In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201206.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,12/1/2020 22:42:51,Gremlins (1984),Blast of Silence (1961),Three Days of the Condor (1975),The Apartment (1960),My Night at Maud's (1969),The Tower (2012),BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?,[Default to Nicolas Cage],"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",OLD-TIMEY PERFECT MURDER,YES IT'S A MUSICAL BUT BEAR WITH ME,UNLIKELY PARTNERS,CULTURAL APPROPRIATION
1,12/1/2020 22:58:50,The Apartment (1960),Three Days of the Condor (1975),Gremlins (1984),Blast of Silence (1961),My Night at Maud's (1969),The Tower (2012),YES IT'S A MUSICAL BUT BEAR WITH ME,OLD-TIMEY PERFECT MURDER,UNLIKELY PARTNERS,CULTURAL APPROPRIATION,"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",[Default to Nicolas Cage],BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?
2,12/1/2020 23:06:32,My Night at Maud's (1969),Blast of Silence (1961),Three Days of the Condor (1975),The Tower (2012),NaN,NaN,UNLIKELY PARTNERS,OLD-TIMEY PERFECT MURDER,"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",YES IT'S A MUSICAL BUT BEAR WITH ME,[Default to Nicolas Cage],BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?,CULTURAL APPROPRIATION
3,12/2/2020 8:02:35,Blast of Silence (1961),The Apartment (1960),My Night at Maud's (1969),Three Days of the Condor (1975),Gremlins (1984),The Tower (2012),"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",YES IT'S A MUSICAL BUT BEAR WITH ME,[Default to Nicolas Cage],BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?,CULTURAL APPROPRIATION,UNLIKELY PARTNERS,OLD-TIMEY PERFECT MURDER
4,12/2/2020 8:21:25,Three Days of the Condor (1975),Gremlins (1984),Blast of Silence (1961),The Tower (2012),NaN,NaN,CULTURAL APPROPRIATION,"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",OLD-TIMEY PERFECT MURDER,UNLIKELY PARTNERS,NaN,NaN,NaN
5,12/3/2020 11:54:00,My Night at Maud's (1969),The Tower (2012),Blast of Silence (1961),The Apartment (1960),Gremlins (1984),NaN,OLD-TIMEY PERFECT MURDER,UNLIKELY PARTNERS,"NEON NOIR: ""HI, MY NAME IS DRIVE. WELCOME TO C...",YES IT'S A MUSICAL BUT BEAR WITH ME,CULTURAL APPROPRIATION,BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Gremlins (1984)', 'Blast of Silence (1961)',
       'Three Days of the Condor (1975)', 'The Apartment (1960)',
       "My Night at Maud's (1969)", 'The Tower (2012)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Gremlins (1984)': <Candidate('Gremlins (1984)')>,
 'Blast of Silence (1961)': <Candidate('Blast of Silence (1961)')>,
 'Three Days of the Condor (1975)': <Candidate('Three Days of the Condor (1975)')>,
 'The Apartment (1960)': <Candidate('The Apartment (1960)')>,
 "My Night at Maud's (1969)": <Candidate('My Night at Maud's (1969)')>,
 'The Tower (2012)': <Candidate('The Tower (2012)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Gremlins (1984), Blast of Silence (1961), Three Days of the Condor (1975), The Apartment (1960), My Night at Maud's (1969), The Tower (2012))>,
 <Ballot(The Apartment (1960), Three Days of the Condor (1975), Gremlins (1984), Blast of Silence (1961), My Night at Maud's (1969), The Tower (2012))>,
 <Ballot(My Night at Maud's (1969), Blast of Silence (1961), Three Days of the Condor (1975), The Tower (2012))>,
 <Ballot(Blast of Silence (1961), The Apartment (1960), My Night at Maud's (1969), Three Days of the Condor (1975), Gremlins (1984), The Tower (2012))>,
 <Ballot(Three Days of the Condor (1975), Gremlins (1984), Blast of Silence (1961), The Tower (2012))>,
 <Ballot(My Night at Maud's (1969), The Tower (2012), Blast of Silence (1961), The Apartment (1960), Gremlins (1984))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
My Night at Maud's (1969)              2  Hopeful
Blast of Silence (1961)                1  Hopeful
Three Days of the Condor (1975)        1  Hopeful
Gremlins (1984)                        1  Hopeful
The Apartment (1960)                   1  Rejected
The Tower (2012)                       0  Rejected

ROUND 2
Candidate                          Votes  Status
-------------------------------  -------  --------
My Night at Maud's (1969)              2  Hopeful
Three Days of the Condor (1975)        2  Hopeful
Blast of Silence (1961)                1  Rejected
Gremlins (1984)                        1  Rejected
The Apartment (1960)                   0  Rejected
The Tower (2012)                       0  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
Three Days of the Condor (1975)        3  Elected
My Night at M

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                                    Votes  Status
---------------------------------------------------------  -------  --------
OLD-TIMEY PERFECT MURDER                                         1  Hopeful
NEON NOIR: "HI, MY NAME IS DRIVE. WELCOME TO CRIME CITY."        1  Hopeful
UNLIKELY PARTNERS                                                1  Hopeful
YES IT'S A MUSICAL BUT BEAR WITH ME                              1  Hopeful
CULTURAL APPROPRIATION                                           1  Hopeful
BAH GOD IS THAT RIP TORN'S MUSIC I HEAR!?                        1  Rejected
[Default to Nicolas Cage]                                        0  Rejected

ROUND 2
Candidate                                                    Votes  Status
---------------------------------------------------------  -------  --------
NEON NOIR: "HI, MY NAME IS DRIVE. WELCOME TO CRIME CITY."        2  Hopeful
OLD-TIMEY PERFECT MURDER                                         1  H